In [1]:
# Import libraries

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

from keras.layers import Dropout
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.constraints import maxnorm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
# To Load dataset
filename = "./Wine Quality/wine.csv"
df = pd.read_csv(filename, index_col='index')
df = df.sample(frac=1).reset_index(drop=True) # Shuffle dataframe
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,6.1,0.59,0.01,2.1,0.056,5.0,13.0,0.99472,3.52,0.56,11.4,5
1,8.3,0.39,0.70,10.6,0.045,33.0,169.0,0.99760,3.09,0.57,9.4,5
2,7.4,0.33,0.26,15.6,0.049,67.0,210.0,0.99907,3.06,0.68,9.5,5
3,7.2,0.39,0.62,11.0,0.047,66.0,178.0,0.99760,3.16,0.50,8.7,5
4,6.9,0.27,0.25,7.5,0.030,18.0,117.0,0.99116,3.09,0.38,13.0,6


In [3]:
# To convert binary classification of quality 
df['quality'] = df['quality'].apply(lambda x: 1 if x > 5 else 0)

In [4]:
df.quality.value_counts()

1    4113
0    2384
Name: quality, dtype: int64

In [5]:
# Creating a test/train split
X = df.iloc[:, 0:11]
y= df.iloc[:, 11]

# Splitting the data set for training and validating 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)

In [6]:
# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train = scaler.transform(X_train)

# Scale the test set
X_test = scaler.transform(X_test)

In [7]:
# Check the shape of the features and the target
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5197, 11)
(1300, 11)
(5197,)
(1300,)


## Tune Batch Size and Number of Epochs

In [8]:
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=11, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100, 150, 200]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.765632 using {'batch_size': 40, 'epochs': 150}
0.756973 (0.007992) with: {'batch_size': 10, 'epochs': 10}
0.761975 (0.010937) with: {'batch_size': 10, 'epochs': 50}
0.763128 (0.015753) with: {'batch_size': 10, 'epochs': 100}
0.760242 (0.015759) with: {'batch_size': 10, 'epochs': 150}
0.760436 (0.011322) with: {'batch_size': 10, 'epochs': 200}
0.751009 (0.006193) with: {'batch_size': 20, 'epochs': 10}
0.764669 (0.009314) with: {'batch_size': 20, 'epochs': 50}
0.759091 (0.004125) with: {'batch_size': 20, 'epochs': 100}
0.765630 (0.014365) with: {'batch_size': 20, 'epochs': 150}
0.762361 (0.006959) with: {'batch_size': 20, 'epochs': 200}
0.736576 (0.009261) with: {'batch_size': 40, 'epochs': 10}
0.764283 (0.014060) with: {'batch_size': 40, 'epochs': 50}
0.760244 (0.009958) with: {'batch_size': 40, 'epochs': 100}
0.765632 (0.007410) with: {'batch_size': 40, 'epochs': 150}
0.764285 (0.007153) with: {'batch_size': 40, 'epochs': 200}
0.749274 (0.014519) with: {'batch_size': 60, 'epoch

## Tune the Training Optimization Algorithm

In [11]:
# Function to create model, required for KerasClassifier
def create_model_optimizer(optimizer='adam'):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=11, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

model = KerasClassifier(build_fn=create_model_optimizer, epochs=100, batch_size=60, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid1 = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result_optimizer = grid1.fit(X_train, y_train) # modify features here
# summarize results
print("Best: %f using %s" % (grid_result_optimizer.best_score_, grid_result_optimizer.best_params_))
means_optimizer = grid_result_optimizer.cv_results_['mean_test_score']
stds_optimizer = grid_result_optimizer.cv_results_['std_test_score']
params_optimizer = grid_result_optimizer.cv_results_['params']
for mean, stdev, param in zip(means_optimizer, stds_optimizer, params_optimizer):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.763711 using {'optimizer': 'Adam'}
0.745046 (0.002613) with: {'optimizer': 'SGD'}
0.751782 (0.009515) with: {'optimizer': 'RMSprop'}
0.674235 (0.022871) with: {'optimizer': 'Adagrad'}
0.541831 (0.073810) with: {'optimizer': 'Adadelta'}
0.763711 (0.006241) with: {'optimizer': 'Adam'}
0.745815 (0.003356) with: {'optimizer': 'Adamax'}
0.758708 (0.012005) with: {'optimizer': 'Nadam'}


## Tune Network Weight Initialization

In [14]:
# Function to create model, required for KerasClassifier
def create_model(init_mode='uniform'):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=11, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(12, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=60, verbose=0)

# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train) # modify features here
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.760246 using {'init_mode': 'glorot_normal'}
0.758707 (0.006469) with: {'init_mode': 'uniform'}
0.760054 (0.006038) with: {'init_mode': 'lecun_uniform'}
0.752164 (0.005091) with: {'init_mode': 'normal'}
0.633055 (0.008740) with: {'init_mode': 'zero'}
0.760246 (0.008273) with: {'init_mode': 'glorot_normal'}
0.754090 (0.004269) with: {'init_mode': 'glorot_uniform'}
0.751781 (0.005597) with: {'init_mode': 'he_normal'}
0.753705 (0.007012) with: {'init_mode': 'he_uniform'}


## Tune the Neuron Activation Function

In [15]:
# Function to create model, required for KerasClassifier
def create_model(activation='relu'):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=11, kernel_initializer='glorot_normal', activation=activation))
    model.add(Dense(12, kernel_initializer='glorot_normal', activation=activation))
    model.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=60, verbose=0)

# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train) # modify features here
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.761207 using {'activation': 'relu'}
0.757745 (0.001181) with: {'activation': 'softmax'}
0.748894 (0.001651) with: {'activation': 'softplus'}
0.758901 (0.008172) with: {'activation': 'softsign'}
0.761207 (0.004832) with: {'activation': 'relu'}
0.757170 (0.008491) with: {'activation': 'tanh'}
0.752742 (0.005054) with: {'activation': 'sigmoid'}
0.751972 (0.005207) with: {'activation': 'hard_sigmoid'}
0.734269 (0.004594) with: {'activation': 'linear'}


## Tune Dropout Regularization

In [21]:
# Function to create model, required for KerasClassifier
def create_model(dropout_rate=0.0, weight_constraint=0):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=11, kernel_initializer='glorot_normal', activation='relu', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dense(12, kernel_initializer='glorot_normal', activation='relu', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=60, verbose=0)

# define the grid search parameters

weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train) # modify features here
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.764672 using {'dropout_rate': 0.0, 'weight_constraint': 1}
0.764672 (0.004076) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.754667 (0.007315) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.762363 (0.002831) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.755438 (0.009625) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.755823 (0.008150) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.756399 (0.007539) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.762750 (0.007811) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.757170 (0.015053) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.758900 (0.001719) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.759862 (0.010183) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.760633 (0.005697) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.763134 (0.008035) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.757939 (0.006306) with: {'dropout_rate': 0.2, 'weight_constraint': 

## Tune the Number of Neurons in the Hidden Layer

In [22]:
# Function to create model, required for KerasClassifier
def create_model(neurons=1):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=11, kernel_initializer='glorot_normal', activation='relu', kernel_constraint=maxnorm(1)))
    model.add(Dense(neurons, kernel_initializer='glorot_normal', activation='relu', kernel_constraint=maxnorm(1)))
    model.add(Dropout(0.0))
    model.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=60, verbose=0)

# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train) # modify features here
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.772563 using {'neurons': 30}
0.670007 (0.046804) with: {'neurons': 1}
0.757361 (0.007195) with: {'neurons': 5}
0.757936 (0.007793) with: {'neurons': 10}
0.762939 (0.006891) with: {'neurons': 15}
0.765633 (0.006703) with: {'neurons': 20}
0.772370 (0.007022) with: {'neurons': 25}
0.772563 (0.006101) with: {'neurons': 30}
